# VGG Transfer Learning for Solar Flare Prediction

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image as pil_image
import imageio
%matplotlib inline
plt.rcParams.update({'font.size': 32})
%config Completer.use_jedi = False

In [46]:
def print_params(model):
    total_params = 0 # initialize counter for total params
    trainable_params = 0 # initialize counter for trainable params
    print('Layer Name\t\tType\t\tFilter shape\t\t# Parameters\tTrainable') # print column headings
    for layer in model.layers: # loop over layers
        lname = layer.name # grab layer name
        ltype = type(layer).__name__ # grab layer type
        ltype[ltype.find('/'):] # parse for only the last part of the string
        if ltype=='Conv2D': # print for convolutional layers
            weights = layer.get_weights()
            print(lname+'\t\t'+ltype+'\t\t'+str(weights[0].shape)+'\t\t'+\
                  str(layer.count_params())+'\t'+str(layer.trainable))
            if layer.trainable:
                trainable_params += layer.count_params()
            total_params += layer.count_params() # update number of params
        elif ltype=='MaxPooling2D': # print for max pool layers
            weights = layer.get_weights()
            print(lname+'\t\t'+ltype+'\t---------------\t\t---')
        elif ltype=='Flatten': # print for flatten layers
            print(lname+'\t\t'+ltype+'\t\t---------------\t\t---')
        elif ltype=='Dense': # print for dense layers
            weights = layer.get_weights()
            print(lname+'\t\t\t'+ltype+'\t\t'+str(weights[0].shape)+'\t\t'+\
                  str(layer.count_params())+'\t'+str(layer.trainable))
            if layer.trainable:
                trainable_params += layer.count_params()
            total_params += layer.count_params() # update number of params
        elif ltype=='Dropout': # print for dropout layers
            print(lname+'\t\t'+ltype+'\t\t------------------\t---')
    print('---------------')
    print('Total trainable parameters: '+str(trainable_params)) # print total params
    print('Total untrainable parameters: '+str(total_params-trainable_params))
    print('Total parameters: '+str(total_params))

In [47]:
def print_shapes(model):
    print('Layer Name\t\tType\t\tInput Shape\t\tOutput Shape\tTrainable')# print column headings
    for layer in model.layers:  # loop over layers
        lname = layer.name # grab layer name
        ltype = type(layer).__name__ # grab layer type
        ltype[ltype.find('/'):] # parse for only the last part of the string
        if ltype=='Conv2D': # print for convolutional layers
            print(lname+'\t\t'+ltype+'\t\t'+str(layer.input_shape)+'\t'+\
                  str(layer.output_shape)+'\t'+str(layer.trainable))
        elif ltype=='MaxPooling2D': # print for maxpool layers
            print(lname+'\t\t'+ltype+'\t'+str(layer.input_shape)+'\t'+\
                  str(layer.output_shape))
        elif ltype=='Flatten': # print for flatten layers
            print(lname+'\t\t'+ltype+'\t\t'+str(layer.input_shape)+'\t'+\
                  str(layer.output_shape))
        elif ltype=='Dense': # print for dense layers
            print(lname+'\t\t\t'+ltype+'\t\t'+str(layer.input_shape)+'\t\t'+\
                  str(layer.output_shape)+'\t'+str(layer.trainable))
        elif ltype=='Dropout': # print for dropout layers
            print(lname+'\t\t'+ltype+'\t\t'+str(layer.input_shape)+'\t'+\
                  str(layer.output_shape))

In [48]:
from keras.applications.vgg16 import preprocess_input
from astropy.io import fits
import skimage.transform

In [49]:
import keras
from keras.layers import Input
input_tensor = Input(shape=(224,224,3))
model1 = keras.applications.vgg16.VGG16(include_top=False,weights='imagenet',input_tensor=input_tensor)

In [50]:
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
new_output = model1.output # take the output as currently defined
new_output = Flatten()(new_output)
#new_output = Dropout(0.5)(new_output)
new_output = Dense(256,activation='relu')(new_output)
new_output = Dropout(0.5)(new_output)
new_output = Dense(256,activation='relu')(new_output)
new_output = Dropout(0.5)(new_output)
new_output = Dense(2,activation='softmax')(new_output) # operate on that output with another dense layer
model2= Model(inputs=model1.input,outputs=new_output) # define a new model with the new output

In [51]:
print_params(model2)

Layer Name		Type		Filter shape		# Parameters	Trainable
block1_conv1		Conv2D		(3, 3, 3, 64)		1792	True
block1_conv2		Conv2D		(3, 3, 64, 64)		36928	True
block1_pool		MaxPooling2D	---------------		---
block2_conv1		Conv2D		(3, 3, 64, 128)		73856	True
block2_conv2		Conv2D		(3, 3, 128, 128)		147584	True
block2_pool		MaxPooling2D	---------------		---
block3_conv1		Conv2D		(3, 3, 128, 256)		295168	True
block3_conv2		Conv2D		(3, 3, 256, 256)		590080	True
block3_conv3		Conv2D		(3, 3, 256, 256)		590080	True
block3_pool		MaxPooling2D	---------------		---
block4_conv1		Conv2D		(3, 3, 256, 512)		1180160	True
block4_conv2		Conv2D		(3, 3, 512, 512)		2359808	True
block4_conv3		Conv2D		(3, 3, 512, 512)		2359808	True
block4_pool		MaxPooling2D	---------------		---
block5_conv1		Conv2D		(3, 3, 512, 512)		2359808	True
block5_conv2		Conv2D		(3, 3, 512, 512)		2359808	True
block5_conv3		Conv2D		(3, 3, 512, 512)		2359808	True
block5_pool		MaxPooling2D	---------------		---
flatten_2		Flatten		---------------		-

In [52]:
for layer in model2.layers[:-3]:
    layer.trainable=False

In [53]:
print_params(model2)

Layer Name		Type		Filter shape		# Parameters	Trainable
block1_conv1		Conv2D		(3, 3, 3, 64)		1792	False
block1_conv2		Conv2D		(3, 3, 64, 64)		36928	False
block1_pool		MaxPooling2D	---------------		---
block2_conv1		Conv2D		(3, 3, 64, 128)		73856	False
block2_conv2		Conv2D		(3, 3, 128, 128)		147584	False
block2_pool		MaxPooling2D	---------------		---
block3_conv1		Conv2D		(3, 3, 128, 256)		295168	False
block3_conv2		Conv2D		(3, 3, 256, 256)		590080	False
block3_conv3		Conv2D		(3, 3, 256, 256)		590080	False
block3_pool		MaxPooling2D	---------------		---
block4_conv1		Conv2D		(3, 3, 256, 512)		1180160	False
block4_conv2		Conv2D		(3, 3, 512, 512)		2359808	False
block4_conv3		Conv2D		(3, 3, 512, 512)		2359808	False
block4_pool		MaxPooling2D	---------------		---
block5_conv1		Conv2D		(3, 3, 512, 512)		2359808	False
block5_conv2		Conv2D		(3, 3, 512, 512)		2359808	False
block5_conv3		Conv2D		(3, 3, 512, 512)		2359808	False
block5_pool		MaxPooling2D	---------------		---
flatten_2		Flatten		-----

## Custom Data Generator

Approach taken from https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3

From https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence: "Sequence are a safer way to do multiprocessing. This structure guarantees that the network will only train once on each sample per epoch which is not the case with generators."

In [54]:
from tensorflow.keras.utils import Sequence
from astropy.io import fits
from keras.applications.vgg16 import preprocess_input
import skimage.transform
class FitsDataGen(Sequence):
    # The input to the data generator will be the dataframe and which columns to use
    def __init__(self, df, X_col, y_col,
                 directory,
                 batch_size,
                 input_size=(224, 224, 3),
                 target_size=None,
                 bitdepth=None,
                 shuffle=True):
        
        self.df = df.copy() # dataframe
        self.X_col = X_col # column for X data (filename)
        self.y_col = y_col # column for y data (class label)
        self.directory = directory # base directory for data
        self.batch_size = batch_size # batch size
        self.input_size = input_size # size expected by network (224,224,3) for VGG
        self.target_size = target_size # resized image for spatial res sims
        self.bitdepth = bitdepth # quantized image for bitdepth sims
        self.shuffle = shuffle # whether to shuffle batches
        
        self.n = len(self.df) # number of data points
        self.nclasses = df[y_col].nunique() # number of classes
            
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path, directory, input_size, target_size, bitdepth):
    
        with fits.open(directory+path) as img: # read in fits image
            img.verify('silentfix')
            img = img[1].data
            
        img = np.expand_dims(img,axis=2) # copy single channel to three to create rgb dimensioned image
        img = np.tile(img,(1,1,3))
        
        # scale to target_size
        if target_size is not None:
            img = skimage.transform.resize(img, (target_size[0],target_size[1]), order=1, mode='reflect',\
                                           clip=True, preserve_range=True, anti_aliasing=True)
        
        # scale to input_size (expected dimensions for input to network)
        img = skimage.transform.resize(img, (input_size[0],input_size[1]), order=1, mode='reflect',\
                                       clip=True, preserve_range=True, anti_aliasing=True)
        
        # put bitdepth stuff here eventually
        
        # scale intensities to range [0,255] as expected by VGG preprocessing function
        # can cheat a bit here and treat each channel the same since these are grayscale images
        img = img + 5978.7 # -5978.7 is minimum of entire magnetogram dataset
        img = img/(2*5978.7)*255 # +5978.7 is maximum of entire magnetogram dataset
#         img[img<-2550] = -2550
#         img = img + 2550 # -5978.7 is minimum of entire magnetogram dataset
#         img = img/(5100)*255 # +5978.7 is maximum of entire magnetogram dataset
#         img[img>255] = 255
        
        
        img = preprocess_input(img) # preprocess according to VGG expectations

        return img
    
    def __get_output(self, label, num_classes):
        return keras.utils.to_categorical(label, num_classes=num_classes)
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        path_batch = batches[self.X_col]
        
        label_batch = batches[self.y_col]

        X_batch = np.asarray([self.__get_input(x, self.directory, self.input_size, self.target_size, self.bitdepth)\
                              for x in path_batch])

        y_batch = np.asarray([self.__get_output(y, self.nclasses) for y in label_batch])
        
        return X_batch, y_batch
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [55]:
train_df = pd.read_csv('train_data.csv',dtype=str)
val_df = pd.read_csv('val_data.csv',dtype=str)
test_df = pd.read_csv('test_data.csv',dtype=str)

In [56]:
train_df = train_df.sample(frac=1).reset_index(drop=True)
val_df = val_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [42]:
train_generator = FitsDataGen(train_df, X_col='filename', y_col='class', directory='Lat60_Lon60_Nans0/',\
                              batch_size=128, input_size=(224,224,3),\
                           target_size=None, bitdepth=None, shuffle=True)
val_generator = FitsDataGen(val_df, X_col='filename', y_col='class', directory='Lat60_Lon60_Nans0/',\
                            batch_size=128, input_size=(224,224,3),\
                           target_size=None, bitdepth=None, shuffle=True)

KeyError: 'class'

## Dataframe approach

In [14]:
# train_df = pd.read_csv('classifier_VGG/Train_Data_by_AR_sr600x600.csv',dtype=str)
# val_df = pd.read_csv('classifier_VGG/Validation_data_by_AR_sr600x600.csv',dtype=str)
# test_df = pd.read_csv('classifier_VGG/Test_Data_by_AR_sr600x600.csv',dtype=str)

In [15]:
# from keras.preprocessing.image import ImageDataGenerator
# from keras.applications.vgg16 import preprocess_input
# from keras.applications.vgg16 import decode_predictions
# train_datagen = ImageDataGenerator(preprocessing_function=scale_spatial_resolution)
# train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,\
#                                                     directory='Lat60_Lon60_Nans0/',\
#                                                     xcol='filename',y_col='class',\
#                                                     target_size=(224,224), color_mode='rgb',\
#                                                     batch_size=128, class_mode='categorical',\
#                                                     shuffle=True)
# val_datagen = ImageDataGenerator(preprocessing_function=scale_spatial_resolution)
# val_generator = val_datagen.flow_from_dataframe(dataframe=val_df,\
#                                                 directory = 'Lat60_Lon60_Nans0/',\
#                                                 xcol='filename',ycol='class',\
#                                                 target_size=(224,224), color_mode='rgb',\
#                                                 batch_size=128, class_mode='categorical',\
#                                                 shuffle=True)

## Dataset approach

In [16]:
# from keras.preprocessing.image import ImageDataGenerator
# from keras.applications.vgg16 import preprocess_input
# from keras.applications.vgg16 import decode_predictions
# train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# train_generator = train_datagen.flow_from_directory('/mnt/solar_flares/AR_Dataset/VGG_Lat60_Lon60_Nans0/TrainingSet_600',\
#                                                     target_size=(224,224), color_mode='rgb',\
#                                                     batch_size=128, class_mode='categorical',\
#                                                     shuffle=True)
# val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# val_generator = val_datagen.flow_from_directory('/mnt/solar_flares/AR_Dataset/VGG_Lat60_Lon60_Nans0/ValidationSet_600',\
#                                                     target_size=(224,224), color_mode='rgb',\
#                                                     batch_size=128, class_mode='categorical',\
#                                                     shuffle=True)

In [17]:
def categorical_tnr(y_true,y_pred):
    import keras.backend as K
    y_true = K.argmax(y_true)
    y_pred = K.argmax(y_pred)
    neg_y_true = 1 - y_true
    neg_y_pred = 1 - y_pred
    fp = K.cast(K.sum(neg_y_true * y_pred),'float32')
    tn = K.cast(K.sum(neg_y_true * neg_y_pred),'float32')
    tnr = tn / (tn + fp + K.epsilon())
    return tnr

def categorical_tpr(y_true,y_pred):
    import keras.backend as K
    y_true = K.argmax(y_true)
    y_pred = K.argmax(y_pred)
    neg_y_pred = 1 - y_pred
    fn = K.cast(K.sum(y_true * neg_y_pred),'float32')
    tp = K.cast(K.sum(y_true * y_pred),'float32')
    tpr = tp / (tp + fn + K.epsilon())
    return tpr

def categorical_tss(y_true,y_pred):
    import keras.backend as K
    tpr = categorical_tpr(y_true,y_pred)
    tnr = categorical_tnr(y_true,y_pred)
    tss = tpr + tnr - 1
    return tss

In [18]:
from keras.optimizers import Adam
adam_opt = Adam(learning_rate=0.001)
model2.compile(loss='categorical_crossentropy', optimizer=adam_opt,\
               metrics=['categorical_accuracy',categorical_tnr,categorical_tpr,categorical_tss])

In [22]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

filepath = 'sr600/model.{epoch:02d}_{val_categorical_tss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_tss', verbose=1, save_best_only=False, mode='max')
early_stop = EarlyStopping(monitor='val_categorical_tss', min_delta=0.001, patience=5, verbose=1, mode='max')
#callbacks_list = [checkpoint, early_stop]
callbacks_list = [checkpoint]

step_size_train = np.ceil(train_generator.n/train_generator.batch_size) 
#step_size_train = 2373 # to help debug val accuracy issue
step_size_val = np.ceil(val_generator.n/val_generator.batch_size)
#step_size_val = 303 # to help debug val accuracy issue

# the following assumes that 0 is the majority class
# class_weights = {0: 1.,
#                  1: (np.asarray(train_generator.classes)==0).sum()/(np.asarray(train_generator.classes)==1).sum()}
# the following taken from https://www.tensorflow.org/tutorials/structured_data/imbalanced_data
#class_weights = {0: (1/(train_generator.classes==0).sum())*len(train_generator.classes)/2,
#                 1: (1/(train_generator.classes==1).sum())*len(train_generator.classes)/2}
# class_weights = {0: 1.,
#                 1: 6.}
class_weights = {0:1., 1: (train_df['class']=='0').sum()/(train_df['class']=='1').sum()}

In [20]:
print(class_weights)

{0: 1.0, 1: 4.0878531849459625}


In [ ]:
model2.fit(train_generator, steps_per_epoch=step_size_train, epochs=10, verbose=1,\
                     callbacks=callbacks_list, validation_data=val_generator, validation_steps=step_size_val,\
                     validation_freq=1, class_weight=class_weights,\
                     max_queue_size=100, workers=20, use_multiprocessing=True)

Epoch 1/10
5933/5933 [==============================] - 17740s 3s/step - loss: 0.9968 - categorical_accuracy: 0.6832 - categorical_tnr: 0.6992 - categorical_tpr: 0.6185 - categorical_tss: 0.3177 - val_loss: 0.4980 - val_categorical_accuracy: 0.8276 - val_categorical_tnr: 0.9489 - val_categorical_tpr: 0.3611 - val_categorical_tss: 0.3099

Epoch 00001: saving model to sr600/model.01_0.31.hdf5
Epoch 2/10
5933/5933 [==============================] - 17760s 3s/step - loss: 0.9159 - categorical_accuracy: 0.7238 - categorical_tnr: 0.7403 - categorical_tpr: 0.6563 - categorical_tss: 0.3966 - val_loss: 0.5230 - val_categorical_accuracy: 0.8239 - val_categorical_tnr: 0.9279 - val_categorical_tpr: 0.4238 - val_categorical_tss: 0.3517

Epoch 00002: saving model to sr600/model.02_0.35.hdf5
Epoch 3/10
5933/5933 [==============================] - 17875s 3s/step - loss: 0.8938 - categorical_accuracy: 0.7359 - categorical_tnr: 0.7528 - categorical_tpr: 0.6666 - categorical_tss: 0.4194 - val_loss: 0.500

Process Keras_worker_ForkPoolWorker-347:
Process Keras_worker_ForkPoolWorker-352:
Process Keras_worker_ForkPoolWorker-351:
Process Keras_worker_ForkPoolWorker-346:
Process Keras_worker_ForkPoolWorker-356:
Process Keras_worker_ForkPoolWorker-349:
Process Keras_worker_ForkPoolWorker-382:
Process Keras_worker_ForkPoolWorker-390:
Process Keras_worker_ForkPoolWorker-344:
Process Keras_worker_ForkPoolWorker-394:
Process Keras_worker_ForkPoolWorker-385:
Process Keras_worker_ForkPoolWorker-393:
Process Keras_worker_ForkPoolWorker-383:
Process Keras_worker_ForkPoolWorker-341:
Process Keras_worker_ForkPoolWorker-391:
Process Keras_worker_ForkPoolWorker-348:
Process Keras_worker_ForkPoolWorker-354:
Process Keras_worker_ForkPoolWorker-400:
Process Keras_worker_ForkPoolWorker-357:
Process Keras_worker_ForkPoolWorker-355:
Process Keras_worker_ForkPoolWorker-396:
Process Keras_worker_ForkPoolWorker-384:
Process Keras_worker_ForkPoolWorker-397:
Process Keras_worker_ForkPoolWorker-358:
Process Keras_wo

In [56]:
model2.fit(train_generator, steps_per_epoch=step_size_train, epochs=10, verbose=1,\
                     callbacks=callbacks_list, validation_data=val_generator, validation_steps=step_size_val,\
                     validation_freq=1, class_weight=class_weights, max_queue_size=100, workers=20, use_multiprocessing=True)

KeyboardInterrupt: 

In [27]:
help(model2.fit)

Help on method fit in module tensorflow.python.keras.engine.training:

fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_batch_size=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False) method of tensorflow.python.keras.engine.functional.Functional instance
    Trains the model for a fixed number of epochs (iterations on a dataset).
    
    Arguments:
        x: Input data. It could be:
          - A Numpy array (or array-like), or a list of arrays
            (in case the model has multiple inputs).
          - A TensorFlow tensor, or a list of tensors
            (in case the model has multiple inputs).
          - A dict mapping input names to the corresponding array/tensors,
            if the model has named inputs.
          - A `tf.data` dataset. Should return

In [15]:
from keras.models import load_model
model2 = load_model('model.best.hdf5', custom_objects={'categorical_tnr': categorical_tnr,\
                                                       'categorical_tpr': categorical_tpr,\
                                                       'categorical_tss': categorical_tss})
model2.compile(loss='categorical_crossentropy', optimizer=adam_opt,\
               metrics=['categorical_accuracy',categorical_tnr,categorical_tpr,categorical_tss])

In [16]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory('VGG_AR_Dataset/TestSet_224',\
                                                target_size=(224,224), color_mode='rgb',\
                                                batch_size=128, class_mode='categorical',\
                                                shuffle=True)

Found 93645 images belonging to 2 classes.


In [17]:
model2.evaluate(test_generator)

732/732 [==============================] - 161s 220ms/step - loss: 0.5478 - categorical_accuracy: 0.7607 - categorical_tnr: 0.7761 - categorical_tpr: 0.6990 - categorical_tss: 0.4751


[0.5457175374031067,
 0.7646430730819702,
 0.7795358300209045,
 0.7046583890914917,
 0.48419439792633057]

In [ ]:
help(model2.evaluate)

In [ ]:
(test_generator.labels==0).sum()

In [ ]:
(test_generator.labels==1).sum()